In [53]:
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

In [ ]:
oasst_ds = Dataset.from_json("./data/oasst_translated.json")
quora_ds = Dataset.from_json("./data/quora_chat_data_translated.json")
wivi_ds = Dataset.from_json("./data/wizard_vicuna_nocode_translated.jsonl")

In [58]:
en_oasst = Dataset.from_json("./data_en/en_oasst.json")
en_wivi = Dataset.from_json("./data_en/wizard_vicuna_nocode.jsonl")
with open("./data_en/quora_chat_data.json") as f:
    quora_en = json.load(f)
quora_en = Dataset.from_list(quora_en)
en_ultra = Dataset.from_parquet('./data_en/en_ultra.parquet')

In [59]:
en_ultra

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 85344
})

In [60]:
def mapper(x, key="text"):
    text = x[key]
    turns = text.split("\n[|")
    messages = []
    for turn in turns:
        if turn.startswith("Human|] "):
            messages.append(
                {"role": "user", "content": turn.replace("Human|] ", "")}
            )
        elif turn.startswith("AI|] "):
            messages.append({"role": "assistant", "content": turn.replace("AI|] ", "")})
        else:
            continue
    return {"messages": [m for m in messages if m["content"] != ""]}

def mapper2(x):
    if x['messages'][0]['role'] != 'user':
        x['messages'] = x['messages'][1:]
    if x['messages'][-1]['role'] != 'assistant':
        x['messages'] = x['messages'][:-1]
    return x

def filter(x):
    messages = x["messages"]
    if len(messages) < 2:
        return False
    current_turn = None
    for message in messages:
        if current_turn is None:
            current_turn = message["role"]
        else:
            if current_turn == message["role"]:
                return False
            else:
                current_turn = message["role"]
    return True


en_oasst = en_oasst.map(mapper).filter(filter).map(mapper2).select_columns(["messages"])
en_wivi = en_wivi.map(lambda x: mapper(x, "conversations")).filter(filter).map(mapper2).select_columns(["messages"])
quora_en = quora_en.map(lambda x: mapper(x, "input")).filter(filter).map(mapper2).select_columns(["messages"])
en_ultra = en_ultra.filter(filter).map(mapper2).select_columns(["messages"])

Map:   0%|          | 0/21164 [00:00<?, ? examples/s]

Map:   0%|          | 0/30468 [00:00<?, ? examples/s]

Map:   0%|          | 0/54456 [00:00<?, ? examples/s]

Filter:   0%|          | 0/54456 [00:00<?, ? examples/s]

Map:   0%|          | 0/54456 [00:00<?, ? examples/s]

Map:   0%|          | 0/85340 [00:00<?, ? examples/s]

In [61]:
en_oasst.to_json('./en_clean_data/en_oasst.jsonl', force_ascii=False)
en_wivi.to_json('./en_clean_data/en_wivi.jsonl', force_ascii=False)
quora_en.to_json('./en_clean_data/en_quora.jsonl', force_ascii=False)
en_ultra.to_json('./en_clean_data/en_ultra.jsonl', force_ascii=False)

Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/55 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/86 [00:00<?, ?ba/s]

365465929

In [ ]:
en_oasst.shuffle()[0]

In [ ]:
oasst_ds.to_json("./clean_data/oasst.jsonl", force_ascii=False)
quora_ds.to_json("./clean_data/quora.jsonl", force_ascii=False)
share_gpt_ds.to_json("./clean_data/share_gpt.jsonl", force_ascii=False)
wivi_ds.to_json("./clean_data/wivi.jsonl", force_ascii=False)

In [ ]:
wivi_ds[0]